<a href="https://colab.research.google.com/github/EleonoraBaim/NPS_Dialogue_system/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [34]:
pip install tqdm

In [1]:
pip install  scikit-multilearn

     |████████████████████████████████| 89 kB 5.4 MB/s 


In [2]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 8.2 MB 9.9 MB/s 


In [14]:
# -*- coding: utf-8 -*-
import pickle
import re
from pathlib import Path
import pandas as pd
import pymorphy2
import json
from sklearn.model_selection import train_test_split
import numpy as np
from skmultilearn.model_selection import IterativeStratification
from collections import Counter
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

pd.set_option("display.max_colwidth", 100)

min_target_size = 1000
project_path = "/content/drive/MyDrive/Colab_Notebooks/NPS_dialogue_system/"
model_path = 'model_parts/'
datatset_folder = 'dataset/'

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


# Functions

In [54]:
def iterative_train_test_split(df, text, label, train_size):
    """Custom iterative train test split which
    'maintains balanced representation with respect
    to order-th label combinations.'
    """
    stratifier = IterativeStratification(n_splits=2, order=1, sample_distribution_per_fold=[1.0-train_size, train_size, ])
    train_indices, test_indices = next(stratifier.split(df[text], df.loc[:, label]))
    X_train, y_train = df[text][train_indices], df.loc[train_indices, label]
    X_test, y_test = df[text][test_indices], df.loc[test_indices, label]
    return X_train, X_test, y_train, y_test


def normalization (text):
  import tqdm

  client_chanel_norm = []
  text_correct = []
  counter = 0

  for i in tqdm.tqdm(text):
    i = str(i).strip().lower()
    i = typos_corr(i)
    text_correct.append(i)

    i = re.sub(r"[^a-zA-Zа-яА-Я ]+", r'', i) 
    i = i.split()
    result = ''
    for word in i:
      result = result + morph.parse(word)[0].normal_form + ' '
    i = result

    client_chanel_norm.append(i)

  return text_correct, client_chanel_norm

def typos_corr(data):
  remove = ['зп', 'КГ']
  for word in remove:
    data = data.replace(word, "kaspigold")
  # Kaspi RED
  remove = ['КР', 'каспи ред', 'kaspi red', 'kaspired', 'каспи рэд', 'каспи red']
  for word in remove:
    data = data.replace(word, "kaspired")
  remove = ['Хор.обсл. ']
  for word in remove:
    data = data.replace(word, "хорошее обслуживание")
  remove = ['ТК']
  for word in remove:
    data = data.replace(word, "торговыйкредит")
  remove = ['%']
  for word in remove:
    data = data.replace(word, "проценты")
  remove = ['КМ']
  for word in remove:
    data = data.replace(word, "каспимагазин")
    
  return data

morph = pymorphy2.MorphAnalyzer()

# 1.0 Data importing

In [8]:
paths = [str(x) for x in Path(project_path + datatset_folder).glob("**/*.xls*")]
datatset = pd.DataFrame()
class_name = []

for file in paths:
    datatset = datatset.append(pd.read_excel(file))


print('Original dataset shape: ' + str(datatset.shape))

Original dataset shape: (230648, 16)


In [9]:
dataset = datatset.copy()

# 2.0 Data preparation

In [10]:
# Working with NA data
print(dataset.isnull().sum())
dataset['Статус звонка'].fillna(0, inplace=True)
dataset['Оценка'].fillna(0, inplace=True)
dataset['Продукт NPS'].fillna('NA', inplace=True)
dataset['Тональность'].fillna('NA', inplace=True)
dataset = dataset[dataset['Статус звонка']==0]
dataset = dataset[dataset['Оценка']!=0]
dataset = dataset[dataset['Продукт NPS']!='NA']
dataset = dataset[dataset['Тональность']!='NA']
print('-----------------------------------')
print('Training dataset shape: ' + str(dataset.shape))

Уникальный идентификатор         0
Продукт опроса                   0
Канал                            0
Операция                     24357
Дата звонка NPS                  0
Статус звонка               196783
Оценка                       14479
Почему                          15
Тональность                  45149
Продукт NPS                  45142
Тематика                     45143
Детализация                  45149
Уточняющая информация       214145
Сегмент NPS                  17904
DEACTIVATED                 179202
UNIQ_ROW                         0
dtype: int64
-----------------------------------
Training dataset shape: (184070, 16)


In [11]:
# Leaving only needed data
dataset = dataset.loc[:,['Уникальный идентификатор','Почему','Продукт опроса', 'Канал', 'Оценка', 'Тональность', 'Продукт NPS', 'Тематика', 'Детализация']]

In [12]:
# Counting num of classes for every comment
num_of_class = pd.DataFrame(dataset['Уникальный идентификатор'].value_counts())
num_of_class.reset_index(inplace=True)
num_of_class = num_of_class.rename(columns = {'Уникальный идентификатор' : 'Количество классов', 'index' : 'Уникальный идентификатор' }, inplace = False)
dataset = dataset.merge(num_of_class, on = 'Уникальный идентификатор', how='left')

# 3.0 Classification labels

In [13]:
# Defining classes, rare classes go to 'nan -> nan -> nan'
dataset.reset_index(inplace = True)

class_name = []
for index, row in dataset.iterrows():
    class_name.append(str(row['Тональность'] + " -> " + str(row['Продукт NPS']) + " -> " + str(row['Тематика']) + " -> " + str(row['Детализация'])))

dataset["target_name"] = pd.Series(class_name)

target_stat = dataset.groupby(['target_name']).size()
target_stat = target_stat[target_stat > min_target_size]

print('\nClasses for learning (small classes will join to nan -> nan -> nan): ')
print(target_stat.sort_values(ascending = False))



Classes for learning (small classes will join to nan -> nan -> nan): 
target_name
Позитив -> Переводы -> Условия -> Удобно переводить деньги                                          11333
Позитив -> Отделения -> Сотрудник -> Хорошо обслужил                                                10122
Позитив -> Платежи на Kaspi.kz -> Услуги к оплате -> Удобно совершать оплату услуг                   4882
Позитив -> Отделения -> Сотрудник -> Быстро решил вопрос клиента                                     4798
Позитив -> Магазин на Kaspi.kz -> Получение -> Нравится доставка...                                  4641
Позитив -> Кредит Наличными -> Получение -> Нравится высокая вероятность одобрения кредита           4492
Позитив -> Kaspi Red -> Условия -> Удобно пользоваться клубной картой                                4364
Позитив -> Кредит Наличными -> Получение -> Нравится быстро и легко получать кредиты                 4323
Позитив -> Kaspi Gold -> Покупки -> Оплата картой через POS – термина

In [16]:
pd.DataFrame(dataset["target_name"].value_counts()).to_excel(str(project_path+model_path+'classes.xlsx'))

In [19]:
categories = {}
categories_exp = {}
for key, value in enumerate(target_stat.index.unique()):
        categories[value] = key
        categories_exp[key] = value

categories['nan -> nan -> nan'] = key +1
categories_exp[key +1] = 'nan -> nan -> nan'

nan_col = key +1

cat_str = json.dumps(categories_exp, ensure_ascii=False)
with open(project_path + model_path + 'categories.json', 'w', encoding='utf-8') as cat_file:
    cat_file.write(cat_str)

print('Saved json file: ' + str(project_path + model_path + 'categories.json'))

target = dataset["target_name"].map(categories)
target.fillna(categories['nan -> nan -> nan'], inplace=True)
dataset['target'] = target


Saved json file: /content/drive/MyDrive/Colab_Notebooks/NPS_dialogue_system/model_parts/categories.json


# 4.0 Binarization of targets

In [20]:
needed_mask = dataset.columns.isin(['Почему', 'Продукт опроса','target'])

needed_df = dataset.iloc[:,needed_mask]

#Delete dublicates
needed = (needed_df.drop_duplicates(subset=needed_df.columns, keep=False)).reset_index()

#Grouping
dataset_m = needed.groupby(['Почему', 'Продукт опроса'])['target'].agg(list)
dataset_m = dataset_m.reset_index()

#Creating Binary matrix
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
multi_target = mlb.fit_transform(dataset_m.target)

dataset_multi=pd.DataFrame()
dataset_multi[['Почему', 'Продукт опроса']] = dataset_m[['Почему', 'Продукт опроса']]
dataset_multi[mlb.classes_] = pd.DataFrame(multi_target)

In [21]:
dataset = dataset_multi.copy()
dataset.head()

,Почему,Продукт опроса,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0
0,"(каз) 5, переплате меньше",АКБ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"(каз) 5. Везде спасает: когда нужно вовремя берем КН и товарный, вчера ПДП сделал КН, но теперь...",9999,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,(каз) 5. МП доступно и легко,СТО,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(каз) Кг хорошая. Легко все, все можно платить с КГ.",ЗП,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(каз) Ни в коем случае не порекомендую. На 4к сделала покупку, добавили 3750 тг плюсом. (бросил...",Kaspi Red,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [27]:
dataset['CONTEXT'] = dataset['Почему'] + " " + dataset['Продукт опроса']
dataset.drop(labels=['Почему', 'Продукт опроса'], axis=1, inplace=True)
cols = dataset.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataset = dataset[cols] 
dataset

,CONTEXT,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0
0,"(каз) 5, переплате меньше АКБ",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"(каз) 5. Везде спасает: когда нужно вовремя берем КН и товарный, вчера ПДП сделал КН, но теперь...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,(каз) 5. МП доступно и легко СТО,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(каз) Кг хорошая. Легко все, все можно платить с КГ. ЗП",0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(каз) Ни в коем случае не порекомендую. На 4к сделала покупку, добавили 3750 тг плюсом. (бросил...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66698,"я хотела уточнить всегда беру кредит когда надо, досрочно закрываю, я хочу что вы скидки сделали...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
66699,"я хочу КР отключить, мне неудобно, когда я хочу оплату совершить в МП картой КГ, выскакивает КР,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
66700,я часто пользуюсь и мне нравится всегда быстро Переводы на Kaspi.kz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
66701,"я че то больше просила, но мало дали, 5: ну быстро как бы. КН ИП",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 5.0 Data Spliting

In [28]:
#Deleting 50% of nan classes in order to decrease samples of this classes 
dataset.drop(index = dataset.loc[dataset[nan_col] == 1].sample(frac=0.5, random_state = 15).index, inplace = True)

text = 'CONTEXT'
labels = dataset.columns[dataset.columns!='CONTEXT']

dataset.reset_index(inplace=True, drop = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
X_train, X_, y_train, y_ = iterative_train_test_split(dataset, text, labels, train_size=0.7)

X_ = pd.DataFrame(X_)
y_ = pd.DataFrame(y_)

dataset_= X_.join(y_)
dataset_.reset_index(inplace=True, drop = True)

X_val, X_test, y_val, y_test = iterative_train_test_split(dataset_, text, labels, train_size=0.5)

train_df = pd.DataFrame(X_train).join(pd.DataFrame(y_train)).reset_index(drop=True)
val_df = pd.DataFrame(X_val).join(pd.DataFrame(y_val)).reset_index(drop=True)
test_df = pd.DataFrame(X_test).join(pd.DataFrame(y_test)).reset_index(drop=True)

In [30]:
train_size = 0.7
val_size = 0.15
test_size = 0.15

# Get counts for each class
counts = {}
counts["train_counts"] = Counter(str(combination) for row in get_combination_wise_output_matrix(
    np.array(y_train), order=1) for combination in row)
counts["val_counts"] = Counter(str(combination) for row in get_combination_wise_output_matrix(
    np.array(y_val), order=1) for combination in row)
counts["test_counts"] = Counter(str(combination) for row in get_combination_wise_output_matrix(
    np.array(y_test), order=1) for combination in row)


# Adjust counts across splits
for k in counts["val_counts"].keys():
    counts["val_counts"][k] = int(counts["val_counts"][k] * \
        (train_size/val_size))
for k in counts["test_counts"].keys():
    counts["test_counts"][k] = int(counts["test_counts"][k] * \
        (train_size/test_size))
    
# View distributions
pd.DataFrame({
    "train": counts["train_counts"],
    "val": counts["val_counts"],
    "test": counts["test_counts"]
}).T.fillna(0)

#del(y_train)
#del(y_validation)

,"(38,)","(11,)","(14,)","(17,)","(1,)","(2,)","(35,)","(12,)","(9,)","(16,)","(19,)","(8,)","(24,)","(7,)","(6,)","(32,)","(33,)","(10,)","(29,)","(20,)","(0,)","(36,)","(26,)","(37,)","(21,)","(28,)","(23,)","(4,)","(15,)","(13,)","(25,)","(18,)","(3,)","(30,)","(27,)","(22,)","(31,)","(34,)","(5,)"
train,9081,572,2323,594,1327,2246,492,1016,745,918,965,858,921,2451,1500,5353,2253,1165,4224,1903,679,651,300,147,770,681,1093,2136,843,2341,787,638,617,1845,2017,865,519,375,475
val,9081,569,2324,592,1330,2244,490,1017,742,919,961,858,924,2454,1502,5352,2254,1166,4228,1904,676,648,298,149,770,653,1092,2137,844,2342,784,634,616,1848,2020,863,522,373,508
test,9081,574,2324,592,1325,2249,494,1017,746,914,966,858,919,2450,1498,5352,2254,1166,4223,1904,681,653,298,144,770,658,1092,2137,844,2342,788,639,620,1843,2016,868,518,378,443


# 6.0 Text preprocessing

In [37]:
# Нормализация текста клиента
print('\nStart text normalization:')

print('Train text ... ')
train_text, norm_train_text = normalization(train_df[text])

print('Test text ... ')
test_text, norm_test_text = normalization(test_df[text])

print('Val text ... ')
val_text, norm_val_text = normalization(val_df[text])

train_df['normalized'] = norm_train_text
test_df['normalized'] = norm_test_text
val_df['normalized'] = norm_val_text



Start text normalization:
Train text ... 


100%|██████████| 37193/37193 [02:09<00:00, 287.65it/s]


Test text ... 


100%|██████████| 8297/8297 [00:26<00:00, 309.06it/s]


Val text ... 


100%|██████████| 8241/8241 [00:26<00:00, 314.66it/s]


In [49]:
cols = train_df.columns.tolist()
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
train_df = train_df[cols] 
test_df = test_df[cols] 
val_df = val_df[cols] 

train_df['CONTEXT'] = train_text
test_df['CONTEXT'] = test_text
val_df['CONTEXT'] = val_text

In [51]:
print('Train: ', train_df.shape)
print('Test: ', test_df.shape)
print('Validation: ', val_df.shape)

Train:  (37193, 41)
Test:  (8297, 41)
Validation:  (8241, 41)


In [53]:
#save datasets
train_df.to_csv(str(project_path + 'ready_datasets/' + 'train_dataset.csv'))
val_df.to_csv(str(project_path + 'ready_datasets/' + 'val_dataset.csv'))
test_df.to_csv(str(project_path + 'ready_datasets/' + 'test_dataset.csv'))